# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [79]:
import sqlite3
import pandas as pd

In [80]:
enron_db = '../../assets/datasets/enron.db'
conn = sqlite3.connect(enron_db)

In [81]:
pd.read_sql("""
    SELECT name FROM sqlite_master
    WHERE type='table'
    ORDER BY name;""",
    con = conn)

,name
0,EmployeeBase
1,MessageBase
2,RecipientBase


In [82]:
pd.read_sql("SELECT * FROM EmployeeBase LIMIT 10", con = conn)

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior
5,6,Don Baughman Jr.,Forestry,ENA East Power,Mgr Trading,Male,Junior
6,7,Sally Beck,Other,Energy Operations,VP,Female,Senior
7,8,Robert Benson,Forestry,ENA East Power,Dir Trading,Male,Senior
8,9,Lynn Blair,Other,ETS,Director,Female,Senior
9,10,Sandra F. Brawner,Forestry,ENA Gas East,Dir Trading,Female,Senior


Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [225]:
sql_lite_master_dataframe = pd.read_sql("""
    SELECT * FROM sqlite_master
    WHERE type = 'table'
    ORDER BY name;""",
    con = conn)

for schema in sql_lite_master_dataframe['sql']:#[sql_lite_master_dataframe['name'] == 'EmployeeBase']['sql']:
    print(schema)

CREATE TABLE EmployeeBase (
                  [eid] INTEGER,
  [name] TEXT,
  [department] TEXT,
  [longdepartment] TEXT,
  [title] TEXT,
  [gender] TEXT,
  [seniority] TEXT
                  
                  )
CREATE TABLE MessageBase (
    mid INTEGER,
    filename TEXT,
    unix_time INTEGER,
    subject TEXT,
    from_eid INTEGER,
    
    PRIMARY KEY(mid ASC),
    FOREIGN KEY(from_eid) REFERENCES Employee(eid)
)
CREATE TABLE RecipientBase (
    mid INTEGER,
    rno INTEGER,
    to_eid INTEGER,
    
    PRIMARY KEY(mid ASC, rno ASC)
    FOREIGN KEY(mid) REFERENCES Message(mid)
    FOREIGN KEY(to_eid) REFERENCES Employee(eid)
)


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [102]:
pd.read_sql("""
    SELECT *
    FROM EmployeeBase
    LIMIT 5
    """, con = conn)

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


Import each of the 3 tables to a Pandas Dataframes

In [103]:
EmployeeBase_dataframe = pd.read_sql("SELECT * FROM EmployeeBase", con = conn)
MessageBase_dataframe = pd.read_sql("SELECT * FROM MessageBase", con = conn)
RecipientBase_dataframe = pd.read_sql("SELECT * FROM RecipientBase", con = conn)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [112]:
print("{} employees".format(EmployeeBase_dataframe.drop_duplicates(subset = 'eid').shape[0]))
print("{} messages".format(MessageBase_dataframe.drop_duplicates(subset = 'mid').shape[0]))

156 employees
21635 messages


In [221]:
MessageBase_dataframe['converted_unix_time'] = pd.to_datetime(MessageBase_dataframe.loc[:, 'unix_time'], unit = 's')
print("The oldest message was sent {}. \nThe newest message was sent {}.".format(min(MessageBase_dataframe['converted_unix_time']), max(MessageBase_dataframe['converted_unix_time'])))

The oldest message was sent 1998-11-13 04:07:00. 
The newest message was sent 2002-06-21 13:37:34.


In [239]:
single_recipient_messages = pd.read_sql("SELECT COUNT(mid) FROM RecipientBase WHERE rno = 1", con = conn)
at_least_5_recipients = pd.read_sql("SELECT COUNT(mid) FROM RecipientBase WHERE rno >= 5", con = conn)
max_recipients_sent_by = pd.read_sql("""
    SELECT EmployeeBase.name 
    FROM EmployeeBase
    JOIN MessageBase ON EmployeeBase.eid = MessageBase.from_eid
    JOIN RecipientBase ON MessageBase.mid = RecipientBase.mid
    WHERE rno = (SELECT MAX(rno) from RecipientBase);"""
    , con = conn).iloc[0][0]

print("{} messages were sent to only one person.".format(single_recipient_messages.values[0][0]))
print("{} messages were sent to at least five people.".format(at_least_5_recipients.values[0][0]))
print("{} send the most messages.".format(max_recipients_sent_by))

21635 messages were sent to only one person.
4162 messages were sent to at least five people.
John J. Lavorato send the most messages.


Rescale to investigate the tail of the curve

In [247]:
pd.read_sql("SELECT rno FROM RecipientBase", con = conn)

from bokeh.charts import Histogram
from bokeh.sampledata.autompg import autompg as df
from bokeh.charts import defaults, vplot, hplot, show, output_file

%matplotlib inline

defaults.width = 450
defaults.height = 350

# input options
hist = Histogram(pd.read_sql("SELECT rno FROM RecipientBase", con = conn), title="number of recipients")
#output_file("histograms.html")

In [248]:
show(
    vplot(
        hplot(hist)
    )
)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead
  app.launch_new_instance()
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: BokehDeprecationWarning: bokeh.io.vplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Column instead
  app.launch_new_instance()


## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [356]:
#Are there more Men or Women employees?
print(EmployeeBase_dataframe[['eid', 'department', 'gender']].groupby(by = 'department')['eid'].count())
#print(EmployeeBase_dataframe['gender'].value_counts(), '\n')
print(pd.read_sql("SELECT gender, COUNT(gender) FROM EmployeeBase GROUP BY gender", con = conn), '\n')

department
Forestry    60
Legal       25
Other       71
Name: eid, dtype: int64
   gender  COUNT(gender)
0  Female             43
1    Male            113 



In [341]:
#How is gender distributed across departments?

#print(EmployeeBase_dataframe.groupby(by = 'department')['gender'].value_counts(), '\n')
print(pd.read_sql("SELECT department, gender, COUNT(gender) FROM EmployeeBase GROUP BY department, gender", con = conn), '\n')

Number of male and female employees by department

  department  gender  COUNT(gender)
0   Forestry  Female             10
1   Forestry    Male             50
2      Legal  Female             13
3      Legal    Male             12
4      Other  Female             20
5      Other    Male             51 



In [342]:
#Who is sending more emails? Men or Women?

print(pd.read_sql("""
    SELECT gender, COUNT(DISTINCT MessageBase.mid)
    FROM EmployeeBase
    JOIN MessageBase ON EmployeeBase.eid = MessageBase.from_eid
    GROUP BY EmployeeBase.gender
    ;""", con = conn), '\n')

Number of emails sent by gender

   gender  COUNT(DISTINCT MessageBase.mid)
0  Female                             8794
1    Male                            12841 



In [343]:
#What's the average number of emails sent by each gender?

average_emails_by_gender = pd.read_sql("""
    SELECT gender, COUNT(DISTINCT MessageBase.mid) / (SELECT COUNT(gender) FROM EmployeeBase GROUP BY gender)
    FROM EmployeeBase
    JOIN MessageBase ON EmployeeBase.eid = MessageBase.from_eid
    GROUP BY EmployeeBase.gender
    ;""", con = conn)
average_emails_by_gender.columns = ['gender', 'emails sent']
print(average_emails_by_gender)

Average emails written by gender

   gender  emails sent
0  Female          204
1    Male          298


In [344]:
#Are there more Juniors or Seniors?

print(pd.read_sql("""
    SELECT seniority, COUNT(seniority) 
    FROM EmployeeBase 
    GROUP BY seniority;"""
    , con = conn))

Number of junior and senior employees

  seniority  COUNT(seniority)
0    Junior                82
1    Senior                74


In [376]:
#Who is sending more emails? Juniors or Seniors?

pd.read_sql("""
    SELECT seniority, COUNT(DISTINCT MessageBase.mid) / (SELECT COUNT(seniority) FROM EmployeeBase GROUP BY seniority) AS emails_sent
    FROM EmployeeBase
    JOIN MessageBase ON EmployeeBase.eid = MessageBase.from_eid
    GROUP BY EmployeeBase.seniority
    ;""", con = conn)
# average_emails_by_seniority.columns = ['seniority', 'emails sent']
# print(average_emails_by_seniority)

,seniority,emails_sent
0,Junior,112
1,Senior,151


In [377]:
#Which department is sending more emails? How does that relate with the number of employees in the department?

print(pd.read_sql("""
    SELECT EmployeeBase.department, COUNT(DISTINCT MessageBase.mid) / COUNT(eid)
    FROM EmployeeBase
    JOIN MessageBase ON EmployeeBase.eid = MessageBase.from_eid
    GROUP BY EmployeeBase.department
    ;""", con = conn))

  department  COUNT(DISTINCT MessageBase.mid) / COUNT(eid)
0   Forestry                                             1
1      Legal                                             1
2      Other                                             1


Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.